In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), ".."))
main_dir = os.path.abspath('..')
os.chdir(main_dir)
sys.path.append(main_dir)

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PhysicsRegression import PhyReg

In [2]:
x_to_fit = []
y_to_fit = []

df = pd.read_csv("./physical/data/Fe X     175.263    2 -   31 3s2 3p5 2P0.5 - 3s2 3p4 3d 2D1.5.csv",)

temperature =   np.array(list(df["temperature"])).  reshape((-1, 1)) / 1e6
edensity =      np.array(list(df["eDensity"])).     reshape((-1, 1)) / 1e10
gofnt =         np.array(list(df["gofnt"])).        reshape((-1, 1)) / 1e-25

x = np.concatenate([temperature, edensity], axis=1)

x_to_fit.append(x)
y_to_fit.append(gofnt)

x_train, x_test, y_train, y_test = train_test_split(x_to_fit[0], y_to_fit[0], test_size=0.15, random_state=2024)
x_to_fit[0] = x_train
y_to_fit[0] = y_train

In [3]:
phyreg = PhyReg(
    path = "./model.pt",
    max_len=10000,
)

phyreg.fit(
    x_to_fit, y_to_fit, 
    use_Divide=True, 
    use_MCTS=False, 
    use_GP=False, 
    use_pysr_init=True, 
    use_const_optimization=False,
    verbose=False,
    oracle_name="physical4",
    oracle_file="./physical/data/oracle_model_case4/",
    oracle_bs=64, oracle_lr=0.006, oracle_epoch=1000,
    use_seperate_type=["id"],
    fixed_x_range=[0.8, 1.5],
    save_oracle_model=True
)

In [5]:
best_gens = phyreg.best_gens
best_gens[0]["predicted_tree"] = "((-13.87596608242056 * 1/((x_1 + 0.8093545090589316))) + 29.20075389899055) * (260003.6830962341 * ((x_0 + -0.313873914775877)** -20.0 * exp((-35.65209487461984 * (((x_0 + 0.27176825338346017)** -2.0))))))"

best_gens = phyreg.constant_optimization(phyreg.best_gens, x_to_fit, y_to_fit)
phyreg.express_skeleton(best_gens, use_sp=True)

idx          : 0
expr skeleton: C_0*(C_1 - C_2/(x_1 + C_3))*exp(C_4/(C_5*x_0 + 1)**C_6)/(x_0 - C_7)**C_8
constants    : 260004.451 227.339 60.217 0.291 -103.763 0.706 3.255 0.036 22.905

